In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.0` // Or use any other 2.x version here
import $ivy.`org.apache.spark::spark-mllib:2.4.0`
import $ivy.`sh.almond::ammonite-spark:0.4.0`
import $ivy.`org.datasyslab:geospark:1.2.0`

import org.apache.spark.serializer.KryoSerializer
import org.apache.spark.storage.StorageLevel
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.rdd.RDD
import org.datasyslab.geospark.enums.{GridType, IndexType}
import org.datasyslab.geospark.spatialOperator.JoinQuery
import org.datasyslab.geospark.formatMapper.shapefileParser.ShapefileReader
import scala.collection.JavaConverters._
import java.io._
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._

val spark = AmmoniteSparkSession.builder()
    .master("local[*]").appName("Validator")
    .getOrCreate()
import spark.implicits._
val appID = spark.sparkContext.applicationId

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                                    

import $ivy.$                                

import $ivy.$                              


import org.apache.spark.serializer.KryoSerializer

import org.apache.spark.storage.StorageLevel

import org.apache.spark.mllib.evaluation.RegressionMetrics

import org.apache.spark.rdd.RDD

import org.datasyslab.geospark.enums.{GridType, IndexType}

import org.datasyslab.geospark.spatialOperator.JoinQuery

import org.datasyslab.geospark.formatMapper.shapefileParser.ShapefileReader

import scala.collection.JavaConverters._

import java.io._

import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@21f44971
import spark.implicits._

appID: String = "local-1557944794618"

### Collecting datasets...
Polygons from several states were collected for both source and target in WKT format. They are available at: https://github.com/aocalderon/RIDIR/tree/master/Datasets/AreaInterpolateValidation.

In [2]:
import sys.process._

val path = "/home/acald013/RIDIR/Datasets/AreaInterpolateValidation"
s"ls -lah ${path}" #| "grep wkt" !

-rw-rw-r-- 1 acald013 acald013  14M May  8 18:47 CA_source.wkt
-rw-rw-r-- 1 acald013 acald013  18M May  8 18:47 CA_target.wkt
-rw-rw-r-- 1 acald013 acald013 1.3M May  8 12:00 MD_source.wkt
-rw-rw-r-- 1 acald013 acald013 1.6M May  8 12:00 MD_target.wkt
-rw-rw-r-- 1 acald013 acald013 2.3M May  8 18:34 NY_source.wkt
-rw-rw-r-- 1 acald013 acald013 4.5M May  8 18:34 NY_target.wkt
-rw-rw-r-- 1 acald013 acald013 4.6M May  8 19:24 PA_source.wkt
-rw-rw-r-- 1 acald013 acald013 7.4M May  8 19:24 PA_target.wkt
-rw-rw-r-- 1 acald013 acald013  12M May 11 19:17 TX_source.wkt
-rw-rw-r-- 1 acald013 acald013  20M May 11 19:17 TX_target.wkt


import sys.process._


path: String = "/home/acald013/RIDIR/Datasets/AreaInterpolateValidation"
res1_2: Int = 0

For each set of source & target, we run the corresponding script:

* For GeoPandas implementation: https://github.com/aocalderon/RIDIR/blob/master/Code/Validation/GeoPandas_interpolate_table_tester.py
* For GeoSpark implementation: https://github.com/aocalderon/RIDIR/blob/master/Code/Areal/src/main/scala/GeoSpark_area_interpolate_tester.scala

Each script save the results to disk for further analysis (files are also available in the same repo).

In [3]:
val path = "/home/acald013/RIDIR/Datasets/AreaInterpolateValidation"
s"ls -lah ${path}" #| "grep tsv" !

-rw-rw-r-- 1 acald013 acald013 105K May  8 19:01 CA_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013 105K May  8 18:49 CA_geospark_test.tsv
-rw-rw-r-- 1 acald013 acald013  13K May  8 17:13 MD_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013  13K May  8 17:05 MD_geospark_test.tsv
-rw-rw-r-- 1 acald013 acald013 290K May 11 18:15 NY_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013 283K May 11 18:14 NY_geospark_test.tsv
-rw-rw-r-- 1 acald013 acald013  37K May  8 19:30 PA_geopandas_test.tsv
-rw-rw-r-- 1 acald013 acald013  37K May  8 19:26 PA_geospark_test.tsv


path: String = "/home/acald013/RIDIR/Datasets/AreaInterpolateValidation"
res2_1: Int = 0

### Set a particular state to run the validation...

In [4]:
val state = "TX"

state: String = "TX"

### Reading results from geopandas implementation...

In [6]:
val geopandas = spark.read.option("header", "false").option("delimiter", "\t").csv(s"${path}/${state}_geopandas_test.tsv")
    .repartition(1).toDF("pandas").orderBy("pandas")
geopandas.show()
geopandas.count()

: 

### Reading results from geospark implementation...

In [ ]:
val geospark = spark.read.option("header", "false").option("delimiter", "\t").csv(s"${path}/${state}_geospark_test.tsv")
    .repartition(1).toDF("spark").orderBy("spark")
geospark.show()
geospark.count()

### Merging both result sets...

In [ ]:
val popPandas = geopandas.rdd.map(_.getString(0).toDouble).sortBy(f => f)
val popSpark  = geospark.rdd.map(_.getString(0).toDouble).sortBy(f => f)

import org.apache.spark.sql.functions._
val pops = popPandas.zip(popSpark)

pops.toDF("pandas", "spark")
    .withColumn("diff", abs($"pandas" - $"spark"))
    .orderBy(desc("diff"))
    .show(50, truncate = false)

### Running some metrics to test similarity...

In [ ]:
val reg = new RegressionMetrics(pops)
println(s"R\u00B2:   ${reg.r2}")
println(s"MAE:  ${reg.meanAbsoluteError}")
println(s"MSE:  ${reg.meanSquaredError}")
println(s"RMSE: ${reg.rootMeanSquaredError}")

In [ ]:
pops